In [3]:
import depthai as dai
import cv2

# Create a pipeline
pipeline = dai.Pipeline()

# Create nodes for left and right cameras
left = pipeline.createMonoCamera()
right = pipeline.createMonoCamera()
left.setBoardSocket(dai.CameraBoardSocket.LEFT)
right.setBoardSocket(dai.CameraBoardSocket.RIGHT)
left.setResolution(dai.MonoCameraProperties.SensorResolution.THE_400_P)
right.setResolution(dai.MonoCameraProperties.SensorResolution.THE_400_P)

# Create XLinkOut nodes to stream the frames to the host
xout_left = pipeline.createXLinkOut()
xout_right = pipeline.createXLinkOut()
xout_left.setStreamName("left")
xout_right.setStreamName("right")

# Link camera outputs to XLinkOut inputs
left.out.link(xout_left.input)
right.out.link(xout_right.input)

# Connect to the device and start the pipeline
with dai.Device(pipeline) as device:
    # Get output queues for both cameras
    q_left = device.getOutputQueue(name="left", maxSize=4, blocking=False)
    q_right = device.getOutputQueue(name="right", maxSize=4, blocking=False)

    while True:
        # Get frames from both cameras
        in_left = q_left.tryGet()
        in_right = q_right.tryGet()

        if in_left is not None:
            frame_left = in_left.getCvFrame()
            cv2.imshow("Left Camera", frame_left)

        if in_right is not None:
            frame_right = in_right.getCvFrame()
            cv2.imshow("Right Camera", frame_right)

        key = cv2.waitKey(1)
        if key == ord('q'):
            break
        elif key == ord('c'):  # Press 'c' to capture and save images
            if in_left is not None:
                cv2.imwrite('captured_image_left.jpg', frame_left)
                print("Left image captured and saved.")
            if in_right is not None:
                cv2.imwrite('captured_image_right.jpg', frame_right)
                print("Right image captured and saved.")

cv2.destroyAllWindows()


<ipython-input-3-2261b444a149>:10: DeprecationWarning: LEFT is deprecated, use CAM_B or address camera by name  instead.
  left.setBoardSocket(dai.CameraBoardSocket.LEFT)
<ipython-input-3-2261b444a149>:11: DeprecationWarning: RIGHT is deprecated, use CAM_C or address camera by name  instead.
  right.setBoardSocket(dai.CameraBoardSocket.RIGHT)


Left image captured and saved.
Right image captured and saved.
Left image captured and saved.
Right image captured and saved.
Left image captured and saved.
Right image captured and saved.
Left image captured and saved.
Right image captured and saved.
Left image captured and saved.
Right image captured and saved.
Left image captured and saved.
Right image captured and saved.
Left image captured and saved.
Right image captured and saved.
Left image captured and saved.
Right image captured and saved.
Left image captured and saved.
Right image captured and saved.
Left image captured and saved.
Right image captured and saved.
Left image captured and saved.
Right image captured and saved.
Left image captured and saved.
Right image captured and saved.
Left image captured and saved.
Right image captured and saved.
Left image captured and saved.
Right image captured and saved.
Left image captured and saved.
Right image captured and saved.
Left image captured and saved.
Right image captured and

In [14]:
import numpy as np
import cv2 as cv
import glob

# Chessboard settings
chessboardSize = (7, 6)
frameSize = (640, 480)
square_size = 20  # Size of a chessboard square [mm]

# Termination criteria for corner sub-pix
criteria = (cv.TERM_CRITERIA_EPS + cv.TERM_CRITERIA_MAX_ITER, 30, 0.001)

# Arrays to store object points and image points from all images.
objpoints = []  # 3D point in real world space
imgpointsL = []  # 2D points in image plane for the left camera
imgpointsR = []  # 2D points in image plane for the right camera

# Prepare object points
objp = np.zeros((chessboardSize[0] * chessboardSize[1], 3), np.float32)
objp[:, :2] = np.mgrid[0:chessboardSize[0], 0:chessboardSize[1]].T.reshape(-1, 2) * square_size

# Directory containing stereo images
pathLeft = 'E:\GSU\Course Work\Computer Vision\Assignment_4\stereoLeft\*.jpg'
pathRight = 'E:\GSU\Course Work\Computer Vision\Assignment_4\stereoRight\*.jpg'

imagesLeft = sorted(glob.glob(pathLeft))
imagesRight = sorted(glob.glob(pathRight))

for imgLeft, imgRight in zip(imagesLeft, imagesRight):
    imgL = cv.imread(imgLeft)
    imgR = cv.imread(imgRight)
    grayL = cv.cvtColor(imgL, cv.COLOR_BGR2GRAY)
    grayR = cv.cvtColor(imgR, cv.COLOR_BGR2GRAY)

    # Find the chess board corners
    retL, cornersL = cv.findChessboardCorners(grayL, chessboardSize, None)
    retR, cornersR = cv.findChessboardCorners(grayR, chessboardSize, None)

    # If found, add object points, refine and add image points
    if retL and retR:
        objpoints.append(objp)
        cornersL = cv.cornerSubPix(grayL, cornersL, (11, 11), (-1, -1), criteria)
        cornersR = cv.cornerSubPix(grayR, cornersR, (11, 11), (-1, -1), criteria)
        imgpointsL.append(cornersL)
        imgpointsR.append(cornersR)

# Calibrate the stereo camera system
retL, cameraMatrixL, distL, rvecsL, tvecsL = cv.calibrateCamera(objpoints, imgpointsL, frameSize, None, None)
retR, cameraMatrixR, distR, rvecsR, tvecsR = cv.calibrateCamera(objpoints, imgpointsR, frameSize, None, None)

flags = cv.CALIB_FIX_INTRINSIC
criteria_stereo = (cv.TERM_CRITERIA_EPS + cv.TERM_CRITERIA_MAX_ITER, 30, 0.001)

# Stereo calibration
retStereo, cameraMatrixL, distL, cameraMatrixR, distR, R, T, E, F = cv.stereoCalibrate(
    objpoints, imgpointsL, imgpointsR, cameraMatrixL, distL, cameraMatrixR, distR, grayL.shape[::-1], criteria_stereo, flags)

# Stereo rectification
rectify_scale = 1  # 0 for no scaling, 1 for full scaling
RL, RR, PL, PR, Q, validRoiL, validRoiR = cv.stereoRectify(cameraMatrixL, distL, cameraMatrixR, distR, grayL.shape[::-1], R, T, None, None, None, None, None, rectify_scale)

# Compute the undistortion and rectification transformation map
mapL1, mapL2 = cv.initUndistortRectifyMap(cameraMatrixL, distL, RL, PL, grayL.shape[::-1], cv.CV_16SC2)
mapR1, mapR2 = cv.initUndistortRectifyMap(cameraMatrixR, distR, RR, PR, grayR.shape[::-1], cv.CV_16SC2)

# Display and save the stereo calibration parameters
print("Stereo Calibration is successful. Parameters are being saved and displayed.")
print(f"Stereo Calibration RMS Error: {retStereo}")
print(f"Left Camera Matrix:\n{cameraMatrixL}")
print(f"Right Camera Matrix:\n{cameraMatrixR}")
print(f"Left Distortion Coefficients:\n{distL}")
print(f"Right Distortion Coefficients:\n{distR}")
print(f"Rotation Matrix between cameras:\n{R}")
print(f"Translation Vector between cameras:\n{T}")

# Save the parameters
cv_file = cv.FileStorage('stereoParams.xml', cv.FILE_STORAGE_WRITE)
cv_file.write("CameraMatrixL", cameraMatrixL)
cv_file.write("CameraMatrixR", cameraMatrixR)
cv_file.write("DistCoeffsL", distL)
cv_file.write("DistCoeffsR", distR)
cv_file.write("RotationMatrix", R)
cv_file.write("TranslationVector", T)
cv_file.write("EssentialMatrix", E)
cv_file.write("FundamentalMatrix", F)
cv_file.release()

print("Parameters saved to 'stereoParams.xml'")


Stereo Calibration is successful. Parameters are being saved and displayed.
Stereo Calibration RMS Error: 0.4757282931190589
Left Camera Matrix:
[[457.7812485    0.         321.28652356]
 [  0.         457.19514736 219.61268967]
 [  0.           0.           1.        ]]
Right Camera Matrix:
[[455.24197497   0.         317.10366051]
 [  0.         450.5034614  204.22159131]
 [  0.           0.           1.        ]]
Left Distortion Coefficients:
[[-1.51416933e-01  4.69484148e+00  4.94664667e-03  1.53374422e-03
  -1.79273275e+01]]
Right Distortion Coefficients:
[[ 0.0479837   0.79993686  0.00443373  0.01964638 -1.39671349]]
Rotation Matrix between cameras:
[[ 9.99990319e-01  2.30553656e-04  4.39422559e-03]
 [-2.57522232e-04  9.99981131e-01  6.13769892e-03]
 [-4.39272761e-03 -6.13877111e-03  9.99971509e-01]]
Translation Vector between cameras:
[[-53.35643152]
 [ -0.10507009]
 [ -4.26484455]]
Parameters saved to 'stereoParams.xml'
